In [28]:
pip install pandas selenium mysql-connector-python sqlalchemy streamlit


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\sivakumar\appdata\local\programs\python\python312\lib\site-packages (2.2.3)




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
import sys
import os

# List of state-specific Redbus links to scrape
state_links = [
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/kaac-transport",
    "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
    
    # ... other links ...
]

# Lists to store scraped data
states = []
routes = []
links = []

# XPaths for scraping
xpath_routes = '//a[@class="route"]'
xpath_pagination_table = '//div[@class="DC_117_paginationTable"]'
xpath_statename = "//h1[@class='D120_search_h1 D120_default' and @id='toc_id_2']"

def initialize_chromedriver():
    try:
        driver = webdriver.Chrome()
        return driver
    except Exception as e:
        print(f"Error initializing WebDriver: {e}")
        sys.exit(1)

def rename_state(text):
    states_data = {
        "Kerala RTC Online Ticket Booking": "Kerala",
        # ... other mappings ...
    }
    return states_data.get(text, text)

def scrape_data(driver, state_links):
    wait = WebDriverWait(driver, 10)
    try:
        for link in state_links:
            driver.get(link)
            driver.maximize_window()
            driver.execute_script("window.scrollBy(0,1500)")
            time.sleep(1)

            page_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{xpath_pagination_table}/div")))
            total_pages = len(page_elements)
            
            state_name = wait.until(EC.presence_of_element_located((By.XPATH, xpath_statename)))
            state = rename_state(state_name.text)

            state_data, routes_data, links_data = [], [], []
            for i in range(1, total_pages + 1):
                objects = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath_routes)))
                for obj in objects:
                    text = obj.get_attribute('text')
                    href = obj.get_attribute('href')
                    routes_data.append(text)
                    links_data.append(href)
                    state_data.append(state)

                if i < total_pages:
                    next_page_xpath = f'//div[@class="DC_117_pageTabs " and text()="{i + 1}"]'
                    nextpage = driver.find_element(By.XPATH, next_page_xpath)
                    nextpage.click()
                    time.sleep(0.2)
                
            routes.extend(routes_data)
            links.extend(links_data)
            states.extend(state_data)
    except Exception as e:
        print(e)
    finally:
        driver.quit()

def export_to_csv(states, routes, links):
    temp_path = r'D:\redbus -REDBUS\01_bus_routes.csv'
    os.makedirs(os.path.dirname(temp_path), exist_ok=True)  # Ensure directory exists
    DATA = pd.DataFrame({'state_names': states, 'routes': routes, 'links': links})
    DATA.to_csv(temp_path, index=False)

def main():
    driver = initialize_chromedriver()
    scrape_data(driver, state_links)
    export_to_csv(states, routes, links)

if __name__ == "__main__":
    main()
